In [1]:
import json
import requests
import pandas as pd
from tqdm import tqdm
import time

In [2]:
KEY_API="RGAPI-d849378c-0d8c-4ea4-88aa-ad296ef67ef1"

In [3]:
headers = {"X-Riot-Token":  KEY_API}

In [4]:
class LolGames:
    
    def __init__(self, apiKey,typeOfGame):
        self.headers = {"X-Riot-Token":  apiKey}
        self.typeOfGame = typeOfGame
    
    def getInfoPlayer(self,player):
      return requests.get(f"https://la1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{player}"
                          , headers=self.headers).json()
    
    def getMatchList(self,account,begin,end):
      return requests.get(f"https://la1.api.riotgames.com/lol/match/v4/matchlists/by-account/{account}?queue={self.typeOfGame}&beginIndex={begin}&endIndex={end}"
                          , headers=self.headers).json()
    
    def getMatch(self,id):
      return requests.get(f"https://la1.api.riotgames.com/lol/match/v4/matches/{id}"
                          , headers=self.headers).json()
    
    def matchList(self,player):
        matchList=[]
        cont=0
        while True:
            r = self.getMatchList(player,cont,cont+100)
            matchList=matchList+r["matches"]
            cont+=100
            if cont >= r["totalGames"]:
                break
        return matchList
    
    def juegos(self,player,champion):
        info=self.getInfoPlayer(player)
        matchList=pd.json_normalize(self.matchList(info["accountId"]))
        matchList=matchList[matchList["champion"]==champion]
        print("Esperando un tiempo ...")
        time.sleep(15)
        games=[]
        for i in tqdm(range(matchList.shape[0])):  
            r=self.getMatch(matchList.iloc[i]["gameId"])
            games.append(r)
            time.sleep(1.5)
        return games

In [5]:
def infoPlayer(name):
    return requests.get(f"https://la1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
                        , headers=headers).json()

In [11]:
def getActualMatch(id):
    return requests.get(f"https://la1.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{id}"
                      , headers=headers).json()


In [12]:
info=infoPlayer("Arodd")

In [13]:
info

{'id': 'AglKCk82x-LJFhnchlCZPKbjcI1VmBVTOtkGPayuHVApNw',
 'accountId': 'JWG187rctiMq0sxtixL2-4zXY0VjUBOMa33y9NAEFb4',
 'puuid': 'Mwo25ZLYasP3MlXtf4oGjN4ePc3ZyIEzqoweFwULn89SawhwekfzwWhhwQ62kwIA941pGr6wcS6yfQ',
 'name': 'Arodd',
 'profileIconId': 588,
 'revisionDate': 1593914702000,
 'summonerLevel': 136}

In [14]:
participants=pd.json_normalize(getActualMatch(info["id"])["participants"])

In [15]:
participants

,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,3,4,86,3887,diamante azul 57,False,lkYpIoDMUYFhbRI6k4AtT-MNgcBra-vOnx2_6AB6fffr,[],"[8010, 9111, 9105, 8014, 8473, 8451, 5008, 500...",8000,8400
1,100,4,7,523,3154,Katastrofé,False,JBWEpZ875Cw_7r4YKT74zFSCiAaeseczE8BXUiFeTesmdg,[],"[8005, 9111, 9104, 8014, 8139, 8135, 5005, 500...",8000,8100
2,100,4,14,117,7,Tarecgosa,False,b33vb8XQOrqeZjyBsUlIq5ZWnUue-4uktp53WarPikg07g,[],"[8214, 8275, 8210, 8236, 8446, 8453, 5008, 500...",8200,8400
3,100,11,4,141,588,Arodd,False,AglKCk82x-LJFhnchlCZPKbjcI1VmBVTOtkGPayuHVApNw,[],"[8128, 8143, 8136, 8105, 8275, 8232, 5008, 500...",8100,8200
4,100,12,4,25,3791,Dregilbird,False,6MF_gfwrXD7--gCx_CGEAcRr5351E8HkUvwWDfTuoWzN6g,[],"[8112, 8126, 8136, 8135, 8345, 8352, 5008, 500...",8100,8300
5,200,4,14,161,709,capitan genial,False,V2sqH0duHam68NWTyXQvZRuK7PXf8-OrgGBDcUNah5nxWA,[],"[8229, 8226, 8210, 8237, 8313, 8347, 5008, 500...",8200,8300
6,200,4,11,104,1154,Kovaky,False,QLogJF9jeZqx7X1aS0uCsa4Jx2aobtx-ZmxRFzy3LFU,[],"[8230, 8275, 8233, 8232, 9104, 8014, 5005, 500...",8200,8000
7,200,7,4,236,1109,MrPorito,False,-aRNORIo1bX0giXopju88BfJwCntrgZpMSRjHXvtk09fpA,[],"[8005, 9111, 9104, 8014, 8138, 8143, 5005, 500...",8000,8100
8,200,4,14,133,552,Alexhiel,False,AkqQaFJWxjQ8jHDYIxViEZRwhzLamkmCrv08YA_XgMwKig,[],"[8005, 8009, 9104, 8014, 8234, 8236, 5005, 500...",8000,8200
9,200,4,14,412,1391,GOSH CRAZY,False,rlSixBrbUQcrYpesF0NjU9etaG6tOSwyGXKs5w1IB1Q5nQ,[],"[8439, 8463, 8429, 8453, 8106, 8138, 5007, 500...",8400,8100


In [16]:
lolInfo=LolGames(KEY_API,420)

In [19]:
for i in range(len(participants["summonerName"])):
    print(participants.iloc[i]["summonerName"])
    games=lolInfo.juegos(participants.iloc[i]["summonerName"],participants.iloc[i]["championId"])
    pd.DataFrame(games).to_json(f"{participants.iloc[i]['summonerName']}.json")

diamante azul 57
Esperando un tiempo ...


100%|██████████| 16/16 [00:35<00:00,  2.22s/it]


Katastrofé 
Esperando un tiempo ...


100%|██████████| 17/17 [00:36<00:00,  2.17s/it]


Tarecgosa
Esperando un tiempo ...


...

100%|██████████| 28/28 [01:07<00:00,  2.42s/it]


Alexhiel
Esperando un tiempo ...


100%|██████████| 199/199 [07:29<00:00,  2.26s/it]


GOSH CRAZY
Esperando un tiempo ...


100%|██████████| 25/25 [00:57<00:00,  2.30s/it]


In [20]:
def getMetricsOfPlayer(df,name):
    aux=pd.DataFrame()
    for i in range(df.shape[0]):
        if df["participants"][i] == None:
            continue
        game=pd.json_normalize(df["participants"][i])
        ident=pd.json_normalize(df["participantIdentities"][i])
        tmp=game.merge(ident,on="participantId")
        aux=aux.append(tmp[tmp["player.summonerName"]==name])
    return aux.groupby("player.summonerName").mean()

In [21]:
final=pd.DataFrame()
for i in participants["summonerName"]:
    print(i)
    df=pd.read_json(f"{i}.json")
    final=final.append(getMetricsOfPlayer(df,i))

diamante azul 57
Katastrofé 
Tarecgosa
Arodd
Dregilbird
capitan genial
Kovaky
MrPorito
Alexhiel
GOSH CRAZY


In [22]:
final

,participantId,teamId,championId,spell1Id,spell2Id,stats.participantId,stats.win,stats.item0,stats.item1,stats.item2,...,timeline.xpPerMinDeltas.30-end,timeline.goldPerMinDeltas.30-end,timeline.csDiffPerMinDeltas.30-end,timeline.xpDiffPerMinDeltas.30-end,timeline.damageTakenPerMinDeltas.30-end,timeline.damageTakenDiffPerMinDeltas.30-end,stats.firstTowerKill,stats.firstTowerAssist,stats.firstInhibitorKill,stats.firstInhibitorAssist
player.summonerName,,,,,,,,,,,,,,,,,,,,,
diamante azul 57,5.812500,156.250000,86.0,11.375000,4.000000,5.812500,0.437500,2503.875000,2745.125000,2865.812500,...,417.800000,431.400000,-2.100000,30.700000,1831.300000,-673.600000,NaN,NaN,NaN,NaN
Katastrofé,4.400000,140.000000,523.0,4.000000,7.000000,4.400000,0.600000,1856.200000,2435.600000,2447.000000,...,527.300000,483.700000,NaN,NaN,995.300000,NaN,NaN,NaN,NaN,NaN
Tarecgosa,6.344444,157.777778,117.0,4.000000,5.922222,6.344444,0.522222,3727.433333,3200.311111,2896.244444,...,513.545614,297.577193,0.061538,12.552564,768.261404,-19.805128,NaN,NaN,NaN,NaN
Arodd,5.575000,151.250000,141.0,11.000000,4.000000,5.575000,0.537500,1878.900000,2926.850000,2822.750000,...,556.888235,413.172549,-0.602222,16.124444,1358.541176,-483.102222,NaN,NaN,NaN,NaN
Dregilbird,5.424460,149.640288,25.0,11.589928,4.000000,5.424460,0.496403,2587.374101,3048.942446,2985.179856,...,549.109314,358.068627,0.023810,1.567857,1062.638235,-6.739286,NaN,NaN,NaN,NaN
capitan genial,5.680851,149.468085,161.0,4.000000,14.335106,5.680851,0.590426,3033.489362,2597.882979,2341.228723,...,437.260976,320.169106,-1.084946,-84.823656,815.131707,-237.224731,NaN,NaN,NaN,NaN
Kovaky,5.832298,157.142857,104.0,4.434783,11.000000,5.832298,0.459627,2083.192547,2743.347826,2928.807453,...,583.614286,451.211429,1.350000,53.423333,1248.334286,-339.110000,NaN,NaN,NaN,NaN
MrPorito,6.892857,171.428571,236.0,7.678571,4.000000,6.892857,0.428571,3074.785714,2641.285714,2488.285714,...,567.255556,461.622222,-0.825000,-55.375000,1295.833333,232.956250,0.142857,0.071429,0.0,0.214286
Alexhiel,5.434343,150.000000,133.0,4.000000,13.530303,5.434343,0.530303,2743.843434,2457.479798,2552.282828,...,605.115972,472.153125,-0.005856,49.950901,1183.426042,-366.211261,NaN,NaN,NaN,NaN


In [23]:
position=["top.100","adc.100","support.100","jungle.100","mid.100","mid.200","jungle.200","adc.200","top.200","support.200"]

In [24]:
final["position"]=position

In [25]:
final.sort_values(by="position")

,participantId,teamId,championId,spell1Id,spell2Id,stats.participantId,stats.win,stats.item0,stats.item1,stats.item2,...,timeline.goldPerMinDeltas.30-end,timeline.csDiffPerMinDeltas.30-end,timeline.xpDiffPerMinDeltas.30-end,timeline.damageTakenPerMinDeltas.30-end,timeline.damageTakenDiffPerMinDeltas.30-end,stats.firstTowerKill,stats.firstTowerAssist,stats.firstInhibitorKill,stats.firstInhibitorAssist,position
player.summonerName,,,,,,,,,,,,,,,,,,,,,
Katastrofé,4.400000,140.000000,523.0,4.000000,7.000000,4.400000,0.600000,1856.200000,2435.600000,2447.000000,...,483.700000,NaN,NaN,995.300000,NaN,NaN,NaN,NaN,NaN,adc.100
MrPorito,6.892857,171.428571,236.0,7.678571,4.000000,6.892857,0.428571,3074.785714,2641.285714,2488.285714,...,461.622222,-0.825000,-55.375000,1295.833333,232.956250,0.142857,0.071429,0.0,0.214286,adc.200
Arodd,5.575000,151.250000,141.0,11.000000,4.000000,5.575000,0.537500,1878.900000,2926.850000,2822.750000,...,413.172549,-0.602222,16.124444,1358.541176,-483.102222,NaN,NaN,NaN,NaN,jungle.100
Kovaky,5.832298,157.142857,104.0,4.434783,11.000000,5.832298,0.459627,2083.192547,2743.347826,2928.807453,...,451.211429,1.350000,53.423333,1248.334286,-339.110000,NaN,NaN,NaN,NaN,jungle.200
Dregilbird,5.424460,149.640288,25.0,11.589928,4.000000,5.424460,0.496403,2587.374101,3048.942446,2985.179856,...,358.068627,0.023810,1.567857,1062.638235,-6.739286,NaN,NaN,NaN,NaN,mid.100
capitan genial,5.680851,149.468085,161.0,4.000000,14.335106,5.680851,0.590426,3033.489362,2597.882979,2341.228723,...,320.169106,-1.084946,-84.823656,815.131707,-237.224731,NaN,NaN,NaN,NaN,mid.200
Tarecgosa,6.344444,157.777778,117.0,4.000000,5.922222,6.344444,0.522222,3727.433333,3200.311111,2896.244444,...,297.577193,0.061538,12.552564,768.261404,-19.805128,NaN,NaN,NaN,NaN,support.100
GOSH CRAZY,7.000000,175.000000,412.0,4.000000,14.000000,7.000000,0.750000,3488.500000,3127.750000,3468.250000,...,271.200000,NaN,NaN,788.000000,NaN,0.000000,0.250000,NaN,NaN,support.200
diamante azul 57,5.812500,156.250000,86.0,11.375000,4.000000,5.812500,0.437500,2503.875000,2745.125000,2865.812500,...,431.400000,-2.100000,30.700000,1831.300000,-673.600000,NaN,NaN,NaN,NaN,top.100


In [26]:
class cleanByGroup:
    def stackGames(self,df):
        arr=[]
        for i in range(10):
            columns=[ f"{df.iloc[i]['position']}.{j}" for j in df.iloc[i].index]
            values=df.iloc[i].values
            data=dict(zip(columns,values))
            arr.append(pd.DataFrame(data,index=[0]))
        return arr
    def posicion(self,role,lane,color):
        if lane=="MIDDLE":
            return f"mid.{color}"
        elif lane=="TOP":
            return f"top.{color}"
        elif lane=="JUNGLE":
            return f"jungle.{color}"
        elif lane=="BOTTOM" and role=="DUO_CARRY":
            return f"adc.{color}"
        elif lane=="BOTTOM" and role=="DUO_SUPPORT":
            return f"support.{color}"
    def processGames(self,df):
        way=['adc.100','adc.200','jungle.100','jungle.200','mid.100','mid.200','support.100','support.200','top.100','top.200']
        final=pd.DataFrame()
        for j in range(len(df.participants)):
            if df.participants[j]==None:
                continue
            test=pd.json_normalize(df.participants[j],max_level=1)
            test["position"]=test.apply(lambda x: self.posicion(x["timeline.role"],x["timeline.lane"],x["teamId"]),axis=1)
            test=test.sort_values(by=['position'])
            if way != list(test["position"]):
                continue
            stacked=self.stackGames(test)
            final=final.append(pd.concat(stacked,axis=1))
        return final

In [27]:
def stackGames(df):
        arr=[]
        for i in range(10):
            columns=[ f"{df.iloc[i]['position']}.{j}" for j in df.iloc[i].index]
            values=df.iloc[i].values
            data=dict(zip(columns,values))
            arr.append(pd.DataFrame(data,index=[0]))
        return arr

In [28]:
X=pd.concat(stackGames(final.sort_values(by="position")),axis=1)

In [29]:
import pickle

In [30]:
loaded_model = pickle.load(open("../class.sav", 'rb'))


In [31]:
best=['adc.100.stats.largestKillingSpree',
 'adc.100.stats.damageDealtToObjectives',
 'adc.100.stats.damageDealtToTurrets',
 'adc.100.stats.turretKills',
 'adc.100.stats.inhibitorKills',
 'adc.200.stats.largestKillingSpree',
 'adc.200.stats.damageDealtToObjectives',
 'adc.200.stats.damageDealtToTurrets',
 'adc.200.stats.turretKills',
 'adc.200.stats.inhibitorKills',
 'jungle.100.stats.deaths',
 'jungle.100.stats.damageDealtToObjectives',
 'jungle.100.stats.damageDealtToTurrets',
 'jungle.100.stats.turretKills',
 'jungle.100.stats.neutralMinionsKilledEnemyJungle',
 'jungle.200.stats.deaths',
 'jungle.200.stats.damageDealtToObjectives',
 'jungle.200.stats.damageDealtToTurrets',
 'jungle.200.stats.turretKills',
 'mid.100.stats.deaths',
 'mid.100.stats.damageDealtToObjectives',
 'mid.100.stats.damageDealtToTurrets',
 'mid.100.stats.turretKills',
 'mid.200.stats.damageDealtToObjectives',
 'mid.200.stats.damageDealtToTurrets',
 'mid.200.stats.turretKills',
 'support.100.stats.deaths',
 'support.100.stats.assists',
 'support.100.stats.damageDealtToObjectives',
 'support.100.stats.damageDealtToTurrets',
 'support.200.stats.assists',
 'support.200.stats.damageDealtToObjectives',
 'support.200.stats.damageDealtToTurrets',
 'top.100.stats.damageDealtToObjectives',
 'top.100.stats.damageDealtToTurrets',
 'top.100.stats.turretKills',
 'top.200.stats.deaths',
 'top.200.stats.damageDealtToObjectives',
 'top.200.stats.damageDealtToTurrets',
 'top.200.stats.turretKills']

In [36]:
X[best].T

,0
adc.100.stats.largestKillingSpree,3.600000
adc.100.stats.damageDealtToObjectives,11307.200000
adc.100.stats.damageDealtToTurrets,2510.400000
adc.100.stats.turretKills,1.200000
adc.100.stats.inhibitorKills,0.200000
adc.200.stats.largestKillingSpree,3.214286
adc.200.stats.damageDealtToObjectives,9940.892857
adc.200.stats.damageDealtToTurrets,3646.464286
adc.200.stats.turretKills,1.071429
adc.200.stats.inhibitorKills,0.035714


In [32]:
loaded_model.predict(X[best])

array([200])

In [33]:
loaded_model.predict_proba(X[best])

array([[0.48722903, 0.51277097]])

In [37]:
loaded_model = pickle.load(open("../no/models/todospartidas.pickle", 'rb'))

In [38]:
loaded_model.predict(X[best])

array([0], dtype=int32)

In [ ]:
loaded_model.predict(X[best])